In [1]:
# 导入所需的库
import pandas as pd
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus

# 加载环境变量
load_dotenv()

try:
    # 数据库连接配置
    DB_USER = os.getenv('DB_USER')
    DB_PASSWORD = quote_plus(os.getenv('DB_PASSWORD'))
    DB_HOST = os.getenv('DB_HOST')
    DB_PORT = os.getenv('DB_PORT')
    DB_NAME = os.getenv('DB_NAME')
    
    # 创建数据库连接URL，使用mysql+pymysql
    db_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    
    # 创建数据库引擎
    engine = create_engine(db_url)
    
    # 测试连接
    with engine.connect() as connection:
        print("数据库连接成功！")
        # 测试查询
        result = connection.execute(text("SELECT DATABASE();"))
        print(f"当前连接的数据库: {result.scalar()}")
        
except Exception as e:
    print(f"连接测试失败: {str(e)}")

数据库连接成功！
当前连接的数据库: data_science


In [2]:
try:
    # 查看表结构
    with engine.connect() as connection:
        result = connection.execute(text("DESCRIBE appstore_dataset;"))
        print("表结构：")
        for row in result:
            print(row)
            
except Exception as e:
    print(f"操作失败: {str(e)}")

表结构：
('app_id', 'varchar(150)', 'YES', '', None, '')
('app_name', 'varchar(300)', 'YES', '', None, '')
('appstore_url', 'text', 'YES', '', None, '')
('primary_genre', 'varchar(50)', 'YES', '', None, '')
('content_rating', 'varchar(20)', 'YES', '', None, '')
('size_bytes', 'bigint', 'YES', '', None, '')
('required_ios_version', 'varchar(10)', 'YES', '', None, '')
('released', 'varchar(30)', 'YES', '', None, '')
('updated', 'varchar(30)', 'YES', '', None, '')
('version', 'varchar(70)', 'YES', '', None, '')
('price', 'decimal(10,2)', 'YES', '', None, '')
('currency', 'varchar(5)', 'YES', '', None, '')
('free', 'tinyint', 'YES', '', None, '')
('developerid', 'bigint', 'YES', '', None, '')
('developer', 'varchar(200)', 'YES', '', None, '')
('developer_url', 'text', 'YES', '', None, '')
('developer_website', 'text', 'YES', '', None, '')
('average_user_rating', 'decimal(3,2)', 'YES', '', None, '')
('reviews', 'bigint', 'YES', '', None, '')
('current_version_score', 'decimal(3,2)', 'YES', '', 

In [3]:
try:
    # 读取CSV文件
    csv_file_path = "/Users/cm/Dataset/appleAppData.csv"
    print(f"正在读取CSV文件: {csv_file_path}")
    df = pd.read_csv(csv_file_path)
    
    # 分析数据
    print("\n数据基本信息：")
    print(f"总行数: {len(df)}")
    
    print("\n各列的最大长度：")
    for column in df.columns:
        if df[column].dtype == 'object':  # 只检查字符串类型的列
            max_length = df[column].astype(str).str.len().max()
            print(f"{column}: {max_length} 字符")
            
    print("\n各列的数据类型：")
    print(df.dtypes)
    
    print("\n各列的空值数量：")
    print(df.isnull().sum())
    
    # 检查content_rating列的唯一值
    print("\ncontent_rating列的唯一值：")
    print(df['Content_Rating'].unique())
    
except Exception as e:
    print(f"操作失败: {str(e)}")

正在读取CSV文件: /Users/cm/Dataset/appleAppData.csv

数据基本信息：
总行数: 1230376

各列的最大长度：
App_Id: 127 字符
App_Name: 253 字符
AppStore_Url: 412 字符
Primary_Genre: 22 字符
Content_Rating: 13 字符
Required_IOS_Version: 7 字符
Released: 20 字符
Updated: 20 字符
Version: 63 字符
Currency: 3 字符
Developer: 163 字符
Developer_Url: 397 字符
Developer_Website: 254 字符

各列的数据类型：
App_Id                      object
App_Name                    object
AppStore_Url                object
Primary_Genre               object
Content_Rating              object
Size_Bytes                 float64
Required_IOS_Version        object
Released                    object
Updated                     object
Version                     object
Price                      float64
Currency                    object
Free                          bool
DeveloperId                  int64
Developer                   object
Developer_Url               object
Developer_Website           object
Average_User_Rating        float64
Reviews                      int

In [10]:
try:
    # 删除旧表
    with engine.connect() as connection:
        connection.execute(text("DROP TABLE IF EXISTS appstore_dataset;"))
        print("成功删除旧表")
    
    # 创建新表
    create_table_sql = """
    CREATE TABLE appstore_dataset (
        app_id VARCHAR(150),
        app_name VARCHAR(300),
        appstore_url TEXT,
        primary_genre VARCHAR(50),
        content_rating VARCHAR(20),
        size_bytes BIGINT,
        required_ios_version VARCHAR(10),
        released VARCHAR(30),
        updated VARCHAR(30),
        version VARCHAR(70),
        price DECIMAL(10,2),
        currency VARCHAR(5),
        free TINYINT,
        developerid BIGINT,
        developer VARCHAR(200),
        developer_url TEXT,
        developer_website TEXT,
        average_user_rating DECIMAL(3,2),
        reviews BIGINT,
        current_version_score DECIMAL(3,2),
        current_version_reviews BIGINT
    ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
    """
    
    with engine.connect() as connection:
        connection.execute(text(create_table_sql))
        print("成功创建新表")
        
    # 读取CSV文件
    csv_file_path = "/Users/cm/Dataset/appleAppData.csv"
    print(f"正在读取CSV文件: {csv_file_path}")
    df = pd.read_csv(csv_file_path)
    
    # 数据预处理
    # 1. 处理空值
    df = df.fillna({
        'App_Name': '',
        'Size_Bytes': 0,
        'Released': '',
        'Price': 0.0,
        'Developer_Url': '',
        'Developer_Website': ''
    })
    
    # 2. 确保数值类型正确
    df['Size_Bytes'] = df['Size_Bytes'].astype('Int64')  # 使用可空的整数类型
    df['Price'] = df['Price'].fillna(0.0)
    df['Free'] = df['Free'].astype(int)
    df['DeveloperId'] = df['DeveloperId'].astype('Int64')
    df['Reviews'] = df['Reviews'].astype('Int64')
    df['Current_Version_Reviews'] = df['Current_Version_Reviews'].astype('Int64')
    
    # 显示数据基本信息
    print(f"\nCSV文件包含 {len(df)} 行数据")
    print("\n数据列名:")
    print(df.columns.tolist())
    
    # 将数据导入到数据库
    print("\n开始导入数据到数据库...")
    
    # 分批导入数据
    batch_size = 10000
    total_rows = len(df)
    
    for i in range(0, total_rows, batch_size):
        batch = df.iloc[i:i+batch_size]
        batch.to_sql('appstore_dataset', engine, if_exists='append', index=False)
        print(f"已导入 {min(i+batch_size, total_rows)}/{total_rows} 条记录")
    
    print("\n数据导入成功！")
    
    # 验证导入的数据
    with engine.connect() as connection:
        result = connection.execute(text("SELECT COUNT(*) FROM appstore_dataset;"))
        count = result.scalar()
        print(f"数据库中的总记录数: {count}")
        
except Exception as e:
    print(f"操作失败: {str(e)}")

成功删除旧表
成功创建新表
正在读取CSV文件: /Users/cm/Dataset/appleAppData.csv

CSV文件包含 1230376 行数据

数据列名:
['App_Id', 'App_Name', 'AppStore_Url', 'Primary_Genre', 'Content_Rating', 'Size_Bytes', 'Required_IOS_Version', 'Released', 'Updated', 'Version', 'Price', 'Currency', 'Free', 'DeveloperId', 'Developer', 'Developer_Url', 'Developer_Website', 'Average_User_Rating', 'Reviews', 'Current_Version_Score', 'Current_Version_Reviews']

开始导入数据到数据库...
已导入 10000/1230376 条记录
已导入 20000/1230376 条记录
已导入 30000/1230376 条记录
已导入 40000/1230376 条记录
已导入 50000/1230376 条记录
已导入 60000/1230376 条记录
已导入 70000/1230376 条记录
已导入 80000/1230376 条记录
已导入 90000/1230376 条记录
已导入 100000/1230376 条记录
已导入 110000/1230376 条记录
已导入 120000/1230376 条记录
已导入 130000/1230376 条记录
已导入 140000/1230376 条记录
已导入 150000/1230376 条记录
已导入 160000/1230376 条记录
已导入 170000/1230376 条记录
已导入 180000/1230376 条记录
已导入 190000/1230376 条记录
已导入 200000/1230376 条记录
已导入 210000/1230376 条记录
已导入 220000/1230376 条记录
已导入 230000/1230376 条记录
已导入 240000/1230376 条记录
已导入 250000/1230376 条记录
已导入 